In [37]:
import sqlite3

# Connect to the database using a properly formatted path.
connection = sqlite3.connect("../databases/takeaway.db")

cursor = connection.cursor()

B1: What is the price distribution of menu items?

In [38]:

# Fetch all results and store them in a variable.
results = cursor.fetchall()

# Print the first 5 fetched results.
print("Sample Data (First 5 Rows):")
#print(results[:10])

# Step 2: Calculate price distribution 
cursor.execute("""
    SELECT 
        CASE 
            WHEN price <= 10 THEN '0-10'
            WHEN price > 10 AND price <= 20 THEN '10-20'
            WHEN price > 20 AND price <= 30 THEN '20-30'
            ELSE '30+'
        END AS price_range,
        COUNT(*) AS item_count
    FROM menuItems
    GROUP BY price_range;
""")

# Fetch the price distribution results.
price_distribution = cursor.fetchall()

# Print the price distribution.
print("\nPrice Distribution:")
for range_, count in price_distribution:
    print(f"Price Range {range_}: {count} items")

# Close the connection when done.
connection.close()


Sample Data (First 5 Rows):

Price Distribution:
Price Range 0-10: 230390 items
Price Range 10-20: 149249 items
Price Range 20-30: 18073 items
Price Range 30+: 5301 items


O2: Which restaurants have the best price-to-rating ratio?

In [31]:
import sqlite3

# Connect to the SQLite database
db_path = "../databases/takeaway.db"
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# SQL query to calculate the best price-to-rating ratio and include address and city
query_price_to_rating_ratio = """
                                         -- Step 1: Calculate the average price for menu items grouped by primarySlug
    WITH AvgMenuPrices AS (
        SELECT 
            primarySlug,
            AVG(price) AS avg_price
        FROM 
            menuItems
        GROUP BY 
            primarySlug
    )
                                        -- Step 2: Join the average prices with restaurant details
    SELECT 
        restaurants.name AS restaurant_name,
        restaurants.address AS restaurant_address,
        restaurants.city AS restaurant_city,
        AvgMenuPrices.avg_price / restaurants.ratings AS price_to_rating_ratio
    FROM 
        restaurants
    JOIN 
        AvgMenuPrices ON restaurants.primarySlug = AvgMenuPrices.primarySlug
    WHERE 
        restaurants.ratings > 0         -- Exclude restaurants with zero or null ratings
    ORDER BY 
        price_to_rating_ratio DESC      -- Sort by the best ratio (highest first)
    LIMIT 10;                           -- Display top 5 restaurants
"""

# Execute the query
cursor.execute(query_price_to_rating_ratio)

# Fetch and display the results
results = cursor.fetchall()
print("Top Restaurants with Best Price-to-Rating Ratio:")
for row in results:
    restaurant_name, restaurant_address, restaurant_city, ratio = row
    #print(f"Restaurant: {restaurant_name}, Address: {restaurant_address}, City: {restaurant_city}, Price-to-Rating Ratio: {ratio:.2f}")
    print(f" {ratio:.2f}   {restaurant_name},  address: {restaurant_address}, {restaurant_city}")
# Close the connection
connection.close()




Top Restaurants with Best Price-to-Rating Ratio:
 9.13   Pizza Hut Delivery,  address: Hubert d'Ydewallestraat 32, Brugge
 8.74   Japans Restaurant Amatsu,  address: Hoogpoort 29, Gent
 8.26   Pizza Hut Delivery,  address: Torhoutsesteenweg 270, Brugge Sint-Andries
 8.20   Wok van Antwerpen,  address: Carnotstraat 123, Antwerpen
 7.85   Mère Liban,  address: Kustlaan 24, Knocke-Heist
 7.80   Drank Express,  address: Kortrijksesteenweg 77, Harelbeke
 7.58   Little Ethiopia,  address: Zirkstraat 8, Antwerpen
 7.45   New Restaurant Henri 1er,  address: Avenue de Messidor 181, Bruxelles
 6.96   D-Afro Flavours,  address: Naamsestraat 87, Leuven
 6.93   Mai Thai,  address: Suikerrui 26, Antwerpen
